In [ ]:
# recommender_non_dl.py
# Requirements:
# pip install numpy pandas scipy scikit-learn requests tqdm

import os
import zipfile
import random
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from math import sqrt
from tqdm import tqdm
import requests
from io import BytesIO

# ------------------------------
# Step 1: Download & Load MovieLens 100k
# ------------------------------
def download_movielens_100k(dest_folder='data'):
    url = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
    os.makedirs(dest_folder, exist_ok=True)
    zip_path = os.path.join(dest_folder, 'ml-100k.zip')
    if not os.path.exists(zip_path):
        print("Downloading MovieLens 100k...")
        r = requests.get(url, stream=True, timeout=60)
        r.raise_for_status()
        with open(zip_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(dest_folder)
    print("Extracted to", dest_folder)

def load_movielens_100k(path='data/ml-100k/u.data'):
    # u.data format: user_id \t item_id \t rating \t timestamp
    names = ['user_id','item_id','rating','timestamp']
    df = pd.read_csv(path, sep='\t', names=names, engine='python')
    return df

# ------------------------------
# Step 2: Preprocess & train/test split (leave-one-out)
# ------------------------------
def leave_one_out_split(ratings_df, seed=42, min_ratings=2):
    """
    For each user with >= min_ratings, hold out one random rating for test.
    Returns train_df, test_df (both pandas DataFrames).
    """
    np.random.seed(seed)
    train_list = []
    test_list = []

    for user, group in ratings_df.groupby('user_id'):
        if len(group) < min_ratings:
            # all to train if not enough ratings
            train_list.append(group)
            continue
        test_idx = np.random.choice(group.index, size=1, replace=False)
        test_list.append(group.loc[test_idx])
        train_list.append(group.drop(test_idx))

    train_df = pd.concat(train_list).reset_index(drop=True)
    test_df = pd.concat(test_list).reset_index(drop=True) if test_list else pd.DataFrame(columns=ratings_df.columns)
    return train_df, test_df

# ------------------------------
# Step 3: Utility - ID mapping & sparse matrix
# ------------------------------
def create_mappings(ratings_df):
    unique_users = ratings_df['user_id'].unique()
    unique_items = ratings_df['item_id'].unique()
    user_to_idx = {u: i for i, u in enumerate(sorted(unique_users))}
    item_to_idx = {i: j for j, i in enumerate(sorted(unique_items))}
    idx_to_user = {i: u for u, i in user_to_idx.items()}
    idx_to_item = {j: i for i, j in item_to_idx.items()}
    return user_to_idx, item_to_idx, idx_to_user, idx_to_item

def build_user_item_matrix(ratings_df, user_to_idx, item_to_idx, shape=None):
    rows = ratings_df['user_id'].map(user_to_idx)
    cols = ratings_df['item_id'].map(item_to_idx)
    data = ratings_df['rating'].astype(float)
    if shape is None:
        shape = (len(user_to_idx), len(item_to_idx))
    mat = csr_matrix((data, (rows, cols)), shape=shape)
    return mat

# ------------------------------
# Step 4: User-User & Item-Item Collaborative Filtering
# ------------------------------
def compute_user_similarity(user_item_csr):
    # returns dense similarity matrix (users x users)
    # for modest datasets (<=10k users) this is OK; for larger, use approximate methods
    print("Computing user-user cosine similarity...")
    # convert to dense or use cosine_similarity on sparse (works)
    sim = cosine_similarity(user_item_csr, dense_output=True)
    np.fill_diagonal(sim, 0.0)
    return sim

def compute_item_similarity(user_item_csr):
    # item vectors are columns, so take transpose
    print("Computing item-item cosine similarity...")
    sim = cosine_similarity(user_item_csr.T, dense_output=True)
    np.fill_diagonal(sim, 0.0)
    return sim

def predict_user_based(user_idx, item_idx, user_item_csr, user_sim, k=20, user_means=None):
    # Weighted sum of k most similar users who rated the item
    if user_means is None:
        user_means = np.zeros(user_item_csr.shape[0])
    sims = user_sim[user_idx]
    # find users who rated the item
    item_col = user_item_csr[:, item_idx].toarray().ravel()
    rated_mask = item_col > 0
    if not rated_mask.any():
        return user_means[user_idx]  # fallback to user mean
    # sort candidate users by similarity
    candidate_idxs = np.where(rated_mask)[0]
    candidate_sims = sims[candidate_idxs]
    top_k_idx = candidate_idxs[np.argsort(candidate_sims)[-k:]][::-1]
    top_sims = sims[top_k_idx]
    top_ratings = item_col[top_k_idx]
    # mean-centering (optional)
    numer = ((top_ratings - user_means[top_k_idx]) * top_sims).sum()
    denom = np.abs(top_sims).sum()
    if denom == 0:
        return user_means[user_idx]
    pred = user_means[user_idx] + numer / denom
    # clip
    return min(5.0, max(1.0, pred))

def predict_item_based(user_idx, item_idx, user_item_csr, item_sim, k=20):
    # Use items user has rated and item-item similarity to target item
    user_row = user_item_csr[user_idx].toarray().ravel()
    rated_mask = user_row > 0
    if not rated_mask.any():
        return user_row.mean() if user_row.size > 0 else 3.0
    candidate_item_idxs = np.where(rated_mask)[0]
    sims = item_sim[item_idx, candidate_item_idxs]
    top_k_idx = candidate_item_idxs[np.argsort(sims)[-k:]][::-1]
    top_sims = sims[np.argsort(sims)[-k:]][::-1]
    top_ratings = user_row[top_k_idx]
    denom = np.abs(top_sims).sum()
    if denom == 0:
        return user_row.mean()
    pred = (top_sims * top_ratings).sum() / denom
    return min(5.0, max(1.0, pred))

# ------------------------------
# Step 5: Matrix Factorization (FunkSVD) via SGD (with biases)
# ------------------------------
class FunkSVD:
    def __init__(self, n_factors=20, lr=0.01, reg=0.02, n_epochs=20, verbose=True, seed=42):
        self.n_factors = n_factors
        self.lr = lr
        self.reg = reg
        self.n_epochs = n_epochs
        self.verbose = verbose
        self.seed = seed

    def fit(self, train_df, n_users, n_items, user_to_idx, item_to_idx):
        np.random.seed(self.seed)
        # Initialize latent factors
        self.P = np.random.normal(scale=0.1, size=(n_users, self.n_factors))
        self.Q = np.random.normal(scale=0.1, size=(n_items, self.n_factors))
        # biases
        self.bu = np.zeros(n_users)
        self.bi = np.zeros(n_items)
        self.mu = train_df['rating'].mean()
        # prepare training triples
        self.train_triples = [
            (user_to_idx[u], item_to_idx[i], float(r))
            for u,i,r in zip(train_df['user_id'], train_df['item_id'], train_df['rating'])
        ]
        # train
        for epoch in range(self.n_epochs):
            random.shuffle(self.train_triples)
            total_loss = 0.0
            for (u, i, r) in self.train_triples:
                pred = self.predict_single(u, i, clip=False)
                err = (r - pred)
                total_loss += err**2
                # update biases
                self.bu[u] += self.lr * (err - self.reg * self.bu[u])
                self.bi[i] += self.lr * (err - self.reg * self.bi[i])
                # update latent factors
                P_u = self.P[u].copy()
                Q_i = self.Q[i].copy()
                self.P[u] += self.lr * (err * Q_i - self.reg * P_u)
                self.Q[i] += self.lr * (err * P_u - self.reg * Q_i)
            rmse_epoch = sqrt(total_loss / len(self.train_triples))
            if self.verbose:
                print(f"Epoch {epoch+1}/{self.n_epochs} RMSE(train)={rmse_epoch:.4f}")
        return self

    def predict_single(self, u_idx, i_idx, clip=True):
        pred = self.mu + self.bu[u_idx] + self.bi[i_idx] + self.P[u_idx].dot(self.Q[i_idx])
        if clip:
            return min(5.0, max(1.0, pred))
        return pred

    def predict(self, user_idx, item_idx):
        return self.predict_single(user_idx, item_idx)

# ------------------------------
# Step 6: Evaluation Metrics
# ------------------------------
def rmse_on_df(model_predict_fn, test_df, user_to_idx, item_to_idx):
    y_true = []
    y_pred = []
    for _, row in test_df.iterrows():
        u, i, r = row['user_id'], row['item_id'], row['rating']
        if u not in user_to_idx or i not in item_to_idx:
            continue
        uidx = user_to_idx[u]
        iidx = item_to_idx[i]
        y_true.append(r)
        y_pred.append(model_predict_fn(uidx, iidx))
    return sqrt(mean_squared_error(y_true, y_pred)) if y_true else None

def precision_recall_at_k(recommend_fn, train_df, test_df, user_to_idx, item_to_idx, K=10):
    """
    For each user in test_df (assumes leave-one-out where each user has one test item),
    generate top-K recommendations excluding items in train for that user.
    """
    # build train lookup per user
    train_by_user = train_df.groupby('user_id')['item_id'].apply(set).to_dict()
    test_by_user = test_df.groupby('user_id')['item_id'].apply(set).to_dict()
    precisions = []
    recalls = []
    users_evaluated = 0

    all_items = set(item_to_idx.keys())

    for user in test_by_user:
        if user not in user_to_idx:
            continue
        uidx = user_to_idx[user]
        train_items = train_by_user.get(user, set())
        candidate_items = sorted(all_items - train_items)  # item ids (original)
        # map candidates to indices
        candidate_idx = [item_to_idx[i] for i in candidate_items if i in item_to_idx]
        # score all candidates
        scores = [(iidx, recommend_fn(uidx, iidx)) for iidx in candidate_idx]
        # top-K by score
        scores.sort(key=lambda x: x[1], reverse=True)
        topk = [iidx for iidx, _ in scores[:K]]
        # map topk indices back to original item ids
        idx_to_item = {v:k for k,v in item_to_idx.items()}
        topk_items = set(idx_to_item[i] for i in topk)
        test_items = test_by_user[user]
        hit_count = len(topk_items & test_items)
        precisions.append(hit_count / K)
        recalls.append(hit_count / len(test_items))
        users_evaluated += 1

    return (np.mean(precisions), np.mean(recalls), users_evaluated)

# ------------------------------
# Step 7: Example pipeline (runs everything)
# ------------------------------
def run_example(download=True):
    # 1) download
    if download:
        try:
            download_movielens_100k()
        except Exception as e:
            print("Download failed:", e)
            print("If offline, ensure data/ml-100k is present.")
    # 2) load
    df = load_movielens_100k(path='data/ml-100k/u.data')
    print("Loaded ratings:", df.shape)
    # 3) split
    train_df, test_df = leave_one_out_split(df, seed=42)
    print("Train size:", len(train_df), "Test size:", len(test_df))
    # 4) mappings
    user_to_idx, item_to_idx, idx_to_user, idx_to_item = create_mappings(pd.concat([train_df, test_df]))
    n_users = len(user_to_idx); n_items = len(item_to_idx)
    print("n_users, n_items:", n_users, n_items)
    # 5) build matrix (train only)
    train_mat = build_user_item_matrix(train_df, user_to_idx, item_to_idx, shape=(n_users, n_items))
    user_means = np.zeros(n_users)
    for u, group in train_df.groupby('user_id'):
        user_means[user_to_idx[u]] = group['rating'].mean()
    # 6) compute similarities (item-based recommended for speed)
    item_sim = compute_item_similarity(train_mat)
    user_sim = compute_user_similarity(train_mat)  # optional

    # 7) define prediction wrappers
    def item_predict_fn(u_idx, i_idx):
        return predict_item_based(u_idx, i_idx, train_mat, item_sim, k=20)

    def user_predict_fn(u_idx, i_idx):
        return predict_user_based(u_idx, i_idx, train_mat, user_sim, k=20, user_means=user_means)

    # 8) Evaluate CF methods by RMSE
    rmse_item = rmse_on_df(item_predict_fn, test_df, user_to_idx, item_to_idx)
    rmse_user = rmse_on_df(user_predict_fn, test_df, user_to_idx, item_to_idx)
    print(f"RMSE Item-based CF: {rmse_item:.4f}")
    print(f"RMSE User-based CF: {rmse_user:.4f}")

    # 9) Precision@K (leave-one-out) for item-based
    prec_item, rec_item, ucount = precision_recall_at_k(item_predict_fn, train_df, test_df, user_to_idx, item_to_idx, K=10)
    print(f"Item-CF Precision@10: {prec_item:.4f} Recall@10: {rec_item:.4f} (on {ucount} users)")

    # 10) Train FunkSVD (matrix factorization) and evaluate
    mf = FunkSVD(n_factors=30, lr=0.005, reg=0.02, n_epochs=15, verbose=True)
    mf.fit(train_df, n_users, n_items, user_to_idx, item_to_idx)
    def mf_predict_fn(u_idx, i_idx):
        return mf.predict(u_idx, i_idx)

    rmse_mf = rmse_on_df(mf_predict_fn, test_df, user_to_idx, item_to_idx)
    print(f"RMSE Matrix Factorization (FunkSVD): {rmse_mf:.4f}")

    prec_mf, rec_mf, _ = precision_recall_at_k(mf_predict_fn, train_df, test_df, user_to_idx, item_to_idx, K=10)
    print(f"MF Precision@10: {prec_mf:.4f} Recall@10: {rec_mf:.4f}")

    # 11) quick example: recommend top-10 for a given user
    example_user_orig_id = list(user_to_idx.keys())[0]
    uidx = user_to_idx[example_user_orig_id]
    candidate_items_idx = list(range(n_items))
    scores = [(iidx, mf_predict_fn(uidx, iidx)) for iidx in candidate_items_idx]
    scores.sort(key=lambda x: x[1], reverse=True)
    top10 = scores[:10]
    print("Top-10 recommendations (item_id,score):")
    for iidx, sc in top10:
        print(idx_to_item[iidx], f"{sc:.3f}")

if __name__ == "__main__":
    run_example(download=True)


Extracted to data
Loaded ratings: (100000, 4)
Train size: 99057 Test size: 943
n_users, n_items: 943 1682
Computing item-item cosine similarity...
Computing user-user cosine similarity...
RMSE Item-based CF: 1.0038
RMSE User-based CF: 1.0053
Item-CF Precision@10: 0.0015 Recall@10: 0.0148 (on 943 users)
Epoch 1/15 RMSE(train)=1.0396
Epoch 2/15 RMSE(train)=0.9739
Epoch 3/15 RMSE(train)=0.9521
Epoch 4/15 RMSE(train)=0.9397
Epoch 5/15 RMSE(train)=0.9310
Epoch 6/15 RMSE(train)=0.9242
Epoch 7/15 RMSE(train)=0.9184
Epoch 8/15 RMSE(train)=0.9130
Epoch 9/15 RMSE(train)=0.9078
Epoch 10/15 RMSE(train)=0.9025
Epoch 11/15 RMSE(train)=0.8969
Epoch 12/15 RMSE(train)=0.8907
Epoch 13/15 RMSE(train)=0.8839
Epoch 14/15 RMSE(train)=0.8765
Epoch 15/15 RMSE(train)=0.8685
RMSE Matrix Factorization (FunkSVD): 0.9882
MF Precision@10: 0.0030 Recall@10: 0.0297
Top-10 recommendations (item_id,score):
169 4.744
100 4.687
50 4.674
178 4.671
357 4.623
408 4.564
64 4.560
190 4.544
483 4.537
114 4.520
